In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dir = '/Users/kevinshuey/massive_dataset/quant_intern/future_price20230628.txt'
df = pd.read_csv(dir, sep='\t', header=0)
df_u = df[['TDATE', 'TTIME', 'UPDATEMILLISEC', 'LASTPX', 'B1', 'S1', 'AVGPX', 'TQ', 'OPENINTS']]
df_u = df_u.dropna(subset=['LASTPX', 'S1', 'B1', 'AVGPX'])
df_u.head()

,TDATE,TTIME,UPDATEMILLISEC,LASTPX,B1,S1,AVGPX,TQ,OPENINTS
0,20230628,85900,4,13125.0,13125.0,13135.0,13125.0,4537,78428
1,20230628,90000,484,13125.0,13120.0,13125.0,13120.0,4551,78425
2,20230628,90000,975,13130.0,13125.0,13135.0,13125.0,4583,78430
3,20230628,90001,476,13125.0,13125.0,13130.0,13125.0,4592,78430
4,20230628,90001,960,13125.0,13120.0,13125.0,13125.0,4601,78435


In [4]:
df_u['CRTPX'] = df['LASTPX'].shift(-1)
df_u['MID'] = (df_u['B1'] + df_u['S1']) / 2

df_u.head()

,TDATE,TTIME,UPDATEMILLISEC,LASTPX,B1,S1,AVGPX,TQ,OPENINTS,CRTPX,MID
0,20230628,85900,4,13125.0,13125.0,13135.0,13125.0,4537,78428,13125.0,13130.0
1,20230628,90000,484,13125.0,13120.0,13125.0,13120.0,4551,78425,13130.0,13122.5
2,20230628,90000,975,13130.0,13125.0,13135.0,13125.0,4583,78430,13125.0,13130.0
3,20230628,90001,476,13125.0,13125.0,13130.0,13125.0,4592,78430,13125.0,13127.5
4,20230628,90001,960,13125.0,13120.0,13125.0,13125.0,4601,78435,13115.0,13122.5


In [5]:
# Stage I: Comparison between P_t and P_t^{mid}
buy_stage1 = df_u['CRTPX'] > df_u['MID']
sell_stage1 = df_u['CRTPX'] < df_u['MID']
equal_stage1 = df_u['CRTPX'] == df_u['MID']
dir1 = np.where(buy_stage1, 'BUY', np.where(sell_stage1, 'SELL', 'EQUAL'))
df_u['dir1'] = dir1
df_u.head()

,TDATE,TTIME,UPDATEMILLISEC,LASTPX,B1,S1,AVGPX,TQ,OPENINTS,CRTPX,MID,dir1
0,20230628,85900,4,13125.0,13125.0,13135.0,13125.0,4537,78428,13125.0,13130.0,SELL
1,20230628,90000,484,13125.0,13120.0,13125.0,13120.0,4551,78425,13130.0,13122.5,BUY
2,20230628,90000,975,13130.0,13125.0,13135.0,13125.0,4583,78430,13125.0,13130.0,SELL
3,20230628,90001,476,13125.0,13125.0,13130.0,13125.0,4592,78430,13125.0,13127.5,SELL
4,20230628,90001,960,13125.0,13120.0,13125.0,13125.0,4601,78435,13115.0,13122.5,SELL


In [6]:
def count(col):
    count1 = df_u[col].value_counts().get('NA1', 0)
    count4 = df_u[col].value_counts().get('NA2', 0)
    count5 = df_u[col].value_counts().get('NA3', 0)
    count2 = df_u[col].value_counts().get('BUY', 0)
    count3 = df_u[col].value_counts().get('SELL', 0)
    count6 = df_u[col].value_counts().get('EQUAL', 0)

    print(f"The number of 'NA1' is {count1}.")
    print(f"The number of 'NA2' is {count4}.")
    print(f"The number of 'NA3' is {count5}.")
    print(f"The number of 'BUY' is {count2}.")
    print(f"The number of 'SELL' is {count3}.")
    print(f"The number of 'EQUAL' is {count6}.")

In [7]:
count('dir1')

The number of 'NA1' is 0.
The number of 'NA2' is 0.
The number of 'NA3' is 0.
The number of 'BUY' is 826600.
The number of 'SELL' is 842045.
The number of 'EQUAL' is 210270.


In [8]:
# Stage II: Comparison between P_{t-1} and P_t^{mid}
df_u['dir2'] = np.where(df_u['dir1'] != 'EQUAL', df_u['dir1'], 'NA1')
df_equal = df_u[df_u['dir1'] == 'EQUAL']

buy_stage2 = (df_u['MID'].loc[df_equal.index] > df_u['LASTPX'].loc[df_equal.index])
sell_stage2 = (df_u['MID'].loc[df_equal.index] < df_u['LASTPX'].loc[df_equal.index])
dir2 = np.where(buy_stage2, 'BUY', np.where(sell_stage2, 'SELL', 'NA2'))
df_u.loc[df_equal.index, 'dir2'] = dir2
df_u.head()

,TDATE,TTIME,UPDATEMILLISEC,LASTPX,B1,S1,AVGPX,TQ,OPENINTS,CRTPX,MID,dir1,dir2
0,20230628,85900,4,13125.0,13125.0,13135.0,13125.0,4537,78428,13125.0,13130.0,SELL,SELL
1,20230628,90000,484,13125.0,13120.0,13125.0,13120.0,4551,78425,13130.0,13122.5,BUY,BUY
2,20230628,90000,975,13130.0,13125.0,13135.0,13125.0,4583,78430,13125.0,13130.0,SELL,SELL
3,20230628,90001,476,13125.0,13125.0,13130.0,13125.0,4592,78430,13125.0,13127.5,SELL,SELL
4,20230628,90001,960,13125.0,13120.0,13125.0,13125.0,4601,78435,13115.0,13122.5,SELL,SELL


In [9]:
count('dir2')

The number of 'NA1' is 0.
The number of 'NA2' is 204577.
The number of 'NA3' is 0.
The number of 'BUY' is 829495.
The number of 'SELL' is 844843.
The number of 'EQUAL' is 0.


In [10]:
# Stage III: Comparison between P_{t} and P_{t-k}
df_u['dP'] = df_u['CRTPX'].diff()

nonzero_indices = df_u.index[df_u['dP'] != 0]
df_u['LASTID'] = np.nan
df_u.loc[nonzero_indices, 'LASTID'] = nonzero_indices
df_u['LASTID'] = df_u['LASTID'].ffill().astype(int)

df_u['LASTVAL'] = df_u['LASTID'].apply(lambda x: df_u.at[x, 'CRTPX'])
df_u.head(30)

,TDATE,TTIME,UPDATEMILLISEC,LASTPX,B1,S1,AVGPX,TQ,OPENINTS,CRTPX,MID,dir1,dir2,dP,LASTID,LASTVAL
0,20230628,85900,4,13125.0,13125.0,13135.0,13125.0,4537,78428,13125.0,13130.0,SELL,SELL,NaN,0,13125.0
1,20230628,90000,484,13125.0,13120.0,13125.0,13120.0,4551,78425,13130.0,13122.5,BUY,BUY,5.0,1,13130.0
2,20230628,90000,975,13130.0,13125.0,13135.0,13125.0,4583,78430,13125.0,13130.0,SELL,SELL,-5.0,2,13125.0
3,20230628,90001,476,13125.0,13125.0,13130.0,13125.0,4592,78430,13125.0,13127.5,SELL,SELL,0.0,2,13125.0
4,20230628,90001,960,13125.0,13120.0,13125.0,13125.0,4601,78435,13115.0,13122.5,SELL,SELL,-10.0,4,13115.0
5,20230628,90002,484,13115.0,13110.0,13120.0,13125.0,4608,78438,13110.0,13115.0,SELL,SELL,-5.0,5,13110.0
6,20230628,90002,941,13110.0,13110.0,13115.0,13120.0,4654,78461,13110.0,13112.5,SELL,SELL,0.0,5,13110.0
7,20230628,90003,481,13110.0,13105.0,13110.0,13120.0,4676,78462,13100.0,13107.5,SELL,SELL,-10.0,7,13100.0
8,20230628,90003,953,13100.0,13100.0,13105.0,13120.0,4691,78460,13105.0,13102.5,BUY,BUY,5.0,8,13105.0
9,20230628,90004,454,13105.0,13100.0,13105.0,13120.0,4699,78459,13100.0,13102.5,SELL,SELL,-5.0,9,13100.0


In [11]:
def Dir3(row):
    if row['dir2'] == 'SELL':
        return 'SELL'
    elif row['dir2'] == 'BUY':
        return 'BUY'
    elif row['dir2'] == 'NA2':
        if row['CRTPX'] > row['LASTVAL']:
            return 'BUY'
        elif row['CRTPX'] < row['LASTVAL']:
            return 'SELL'
        else:
            return 'NA3'
    else:
        return 'NA3'
# 'apply' methods will consume a little more time
df_u['dir3'] = df_u.apply(Dir3, axis=1)

In [12]:
df_u.head(60)

,TDATE,TTIME,UPDATEMILLISEC,LASTPX,B1,S1,AVGPX,TQ,OPENINTS,CRTPX,MID,dir1,dir2,dP,LASTID,LASTVAL,dir3
0,20230628,85900,4,13125.0,13125.0,13135.0,13125.0,4537,78428,13125.0,13130.0,SELL,SELL,NaN,0,13125.0,SELL
1,20230628,90000,484,13125.0,13120.0,13125.0,13120.0,4551,78425,13130.0,13122.5,BUY,BUY,5.0,1,13130.0,BUY
2,20230628,90000,975,13130.0,13125.0,13135.0,13125.0,4583,78430,13125.0,13130.0,SELL,SELL,-5.0,2,13125.0,SELL
3,20230628,90001,476,13125.0,13125.0,13130.0,13125.0,4592,78430,13125.0,13127.5,SELL,SELL,0.0,2,13125.0,SELL
4,20230628,90001,960,13125.0,13120.0,13125.0,13125.0,4601,78435,13115.0,13122.5,SELL,SELL,-10.0,4,13115.0,SELL
5,20230628,90002,484,13115.0,13110.0,13120.0,13125.0,4608,78438,13110.0,13115.0,SELL,SELL,-5.0,5,13110.0,SELL
6,20230628,90002,941,13110.0,13110.0,13115.0,13120.0,4654,78461,13110.0,13112.5,SELL,SELL,0.0,5,13110.0,SELL
7,20230628,90003,481,13110.0,13105.0,13110.0,13120.0,4676,78462,13100.0,13107.5,SELL,SELL,-10.0,7,13100.0,SELL
8,20230628,90003,953,13100.0,13100.0,13105.0,13120.0,4691,78460,13105.0,13102.5,BUY,BUY,5.0,8,13105.0,BUY
9,20230628,90004,454,13105.0,13100.0,13105.0,13120.0,4699,78459,13100.0,13102.5,SELL,SELL,-5.0,9,13100.0,SELL


In [13]:
count('dir3')

The number of 'NA1' is 0.
The number of 'NA2' is 0.
The number of 'NA3' is 204577.
The number of 'BUY' is 829495.
The number of 'SELL' is 844843.
The number of 'EQUAL' is 0.


In [14]:
count_na3 = (df_u['dir3'] == 'NA3').sum()
count_buy = (df_u['dir3'] == 'BUY').sum()
count_sell = (df_u['dir3'] == 'SELL').sum()

counts = {
    'Category': ['BUY', 'SELL', 'N/A'],
    'Count': [count_buy, count_sell, count_na3]
}

df_counts = pd.DataFrame(counts)
df_counts

,Category,Count
0,BUY,829495
1,SELL,844843
2,N/A,204577


In [15]:
print(df_counts)

  Category   Count
0      BUY  829495
1     SELL  844843
2      N/A  204577
